In [ ]:
%matplotlib inline
%gui qt
import warnings
warnings.filterwarnings('ignore')
from spectral_cube import SpectralCube
import spectral_cube
import pyregion
from glue import qglue
#spectral_cube.version.version

In [ ]:
nro_13co_cube = SpectralCube.read('../nro_maps/13CO_20160904_FOREST_xyb_spheroidal_0254kms_YS.fits')
nro_12co_cube = SpectralCube.read('../nro_maps/12CO_20160905_BEARS_FOREST_xyb_spheroidal_0254kms_YS.fits')
ds9_file = '../nro_maps/SouthShells.reg'

In [ ]:
nro_12co_cube[100,:,:].quicklook()

### Glue Visualization of $^{12}$CO and $^{13}$CO Full Cubes

In [ ]:
nro_12co_data = nro_12co_cube.unmasked_data[:]
nro_13co_data = nro_13co_cube.unmasked_data[:]
#app_full=qglue(nro_12co=nro_12co_data, nro_13co=nro_13co_data)


### Extracting subcubes around shell candidates.
Using spectral-cube and pyregion python packages with ds9 region file of circles around shell candidates.


http://spectral-cube.readthedocs.io/en/latest/manipulating.html#extracting-a-subcube-from-a-ds9-region

In [ ]:
import pyregion
import copy
region_file = '../nro_maps/SouthShells.reg'
region_list = pyregion.open('../nro_maps/SouthShells.reg')

In [ ]:
def subcubes_from_ds9(cube, region_file='../nro_maps/SouthShells.reg', pad_factor=1., shape='exact'):
    """
    Parameters
    ----------
    cube: SpectralCube
        The cube to be chopped. Must be type spectral_cube.SpectralCube.
    region_file: str
        Path to a ds9 region file.
    pad_factor: float, optional
        Expand the subcube around the region by this factor.
    shape: {'square', 'exact'}
        The shape of the subcube returned. 'square' returns the
        smallest square subcube that contains the region.
        'exact' returns only the pixels contained within the region.
    
    Returns
    --------
    subcubes: list of SpectralCube of SpectralCube
    """
    from spectral_cube import SpectralCube
    import pyregion

    if shape == 'square':
        import astropy.units as u
        subcube_list = []
        region_list = pyregion.open(region_file)
        for region in region_list:
            half_width = region.coord_list[2] * pad_factor * u.deg
            ra_center = region.coord_list[0] * u.deg
            dec_center = region.coord_list[1] * u.deg
            ra_range = [ra_center - half_width, ra_center + half_width]
            dec_range = [dec_center - half_width, dec_center + half_width]
            #print(ra_range, dec_range)
            subcube_list.append(cube.subcube(ra_range[1], ra_range[0], dec_range[0], dec_range[1]))
    if shape == 'exact':
        region_list = pyregion.open(region_file)
        subcube_list = []
        for region in region_list:
            
            if pad_factor != 1.:
                new_string = '{};{}({},{},{}")'.format(region.coord_format, region.name,
                                region.coord_list[0], region.coord_list[1],
                                region.coord_list[2]*3600.*pad_factor)
                region = pyregion.parse(new_string)[0]
                
            subcube_list.append(cube.subcube_from_ds9region(pyregion.ShapeList([region])))
    if len(subcube_list) == 1:
        return subcube_list[0]
    else:
        return subcube_list


### Plot shell candidates


In [ ]:
import aplpy
from glue.app.qt.application import GlueApplication
nro_12co_subcube_list = subcubes_from_ds9(nro_12co_cube, shape='square', pad_factor=1.2)
#mom0_12co = nro_12co_subcube_list[5].moment(order=0)
#f = aplpy.FITSFigure(nro_12co_subcube_list[1][110].hdu)
#f.show_colorscale(cmap='gray')
#f.show_colorbar()
#glue_subcubes=qglue()
nro_12co_subcube_list[100,:,:].quicklook()

### Send Shell Candidates to Glue

In [8]:
for subcube in nro_12co_subcube_list:
    subcube.to_glue()
    
#nro_12co_subcube_list[1].to_glue(name='shell_2')

  File "//anaconda/lib/python3.4/runpy.py", line 170, in _run_module_as_main
    "__main__", mod_spec)
  File "//anaconda/lib/python3.4/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "//anaconda/lib/python3.4/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "//anaconda/lib/python3.4/site-packages/traitlets/config/application.py", line 596, in launch_instance
    app.start()
  File "//anaconda/lib/python3.4/site-packages/ipykernel/kernelapp.py", line 442, in start
    ioloop.IOLoop.instance().start()
  File "//anaconda/lib/python3.4/site-packages/zmq/eventloop/ioloop.py", line 162, in start
    super(ZMQIOLoop, self).start()
  File "//anaconda/lib/python3.4/site-packages/tornado/ioloop.py", line 831, in start
    self._run_callback(callback)
  File "//anaconda/lib/python3.4/site-packages/tornado/ioloop.py", line 604, in _run_callback
    ret = callback()
  File "//anaconda/lib/python3.4/site-packages/tornado/stack_contex

In [ ]:
#a??

In [ ]:
#nro_12co_subcube_list[2].to_glue(name='shell_3', glue_app=a)

#f.save('moment_0.png')

### Return sources from Spitzer Orion fits table that fall within each shell subcube.

In [ ]:
from astropy.io import fits
spitzer_orion_file = '../catalogs/spitzer_orion.fits'
f = fits.open(spitzer_orion_file)
#f[1].fields
#yso_list = []
#for subcube in nro_12co_subcube_list:
#   
#    yso_list.append(yso)

### Extract Spitzer postage stamps around shell candidates.